In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))

# FASE 1: Análisis

**1. IMPORTACIÓN DE LIBRERIAS**

In [37]:
import pandas as pd
import numpy as np

**2. CARGA DE LOS DATOS**

A continuación se declara el siguiente dataframe generado en pandas y cargado a través de un archivo en csv llamado **dataset_covid2019.csv** en la ruta relativa "../../data/dataset_covid2019.csv", donde están contenidas las variables categóricas y numéricas. 

In [4]:
df = pd.read_csv("../../data/dataset_covid2019.csv", encoding = "ISO-8859-1")

In [5]:
#Función de la librería para que muestre todas las columnas
pd.options.display.max_columns = None

In [6]:
df.head(3)

,Fecha_Actualizacion,Id_Registro,Origen,Sector,Entidad_Um,Sexo,Entidad_Nac,Entidad_Res,Municipio_Res,Tipo_Paciente,Fecha_Ingreso,Fecha_Sintomas,Fecha_Def,Intubado,Neumonia,Edad,Nacionalidad,Embarazo,Habla_Lengua_Indig,Diabetes,Epoc,Asma,Inmusupr,Hipertension,Otra_Com,Cardiovascular,Obesidad,Renal_Cronica,Tabaquismo,Otro_Caso,Resultado,Migrante,Pais_Nacionalidad,Pais_Origen,Uci,Municipio,Estado,Latitud,Longitud
0,04/05/2020,1ab4e5,2,Publico,25,Hombre,25,25,6,2,06/04/2020,31/03/2020,10/04/2020,Si,Si,42,1,No aplica,2,Si,No,No,No,Si,No,No,Si,No,No,2,Positivo,99,MÃÂ©xico,99,No,Culiacan,Sinaloa,25.0,-107.5
1,04/05/2020,1448c7,2,Publico,25,Hombre,99,25,6,2,14/04/2020,04/04/2020,17/04/2020,Si,Si,45,2,No aplica,2,No,No,No,No,No,No,No,No,No,No,2,Positivo,2,Estados Unidos de AmÃÂ©rica,99,No,Culiacan,Sinaloa,25.0,-107.5
2,04/05/2020,73442,2,Publico,25,Mujer,25,25,6,2,14/04/2020,06/04/2020,9999-99-99,No,Si,50,1,No,2,No,No,No,No,No,No,No,Si,No,No,2,Positivo,99,MÃÂ©xico,99,No,Culiacan,Sinaloa,25.0,-107.5


**3. ANÁLISIS DESCRIPTIVO**

Ante el desconocimiento del campo a tratar se realizan las siguientes observaciones a modo exploratorio con el fin de obtener un primer acercamiento a los datos y a la información.

**Número de personas que han dado positivo VS negativo**

El siguiente resultado representa la cifra de positivos y negativos confirmados a 04 de Mayo de 2020,de un total de **100039 pacientes**.

In [7]:
df['Resultado'].value_counts()

Negativo               61992
Positivo               24904
Resultado pendiente    13143
Name: Resultado, dtype: int64

Se observa que el número de positivos confirmados representan el **24,89%** de los usuarios, con un total de 24.904 casos. Se destaca que una parte considerable de los usuarios, un 13%, permanece con resultados pendientes, lo que puede ser un indicio del desborde de la siatuación. Los casos negativos representan la mayor parte, siendo el 61,97% de los pacientes.

**Numero de personas en UCI**


A continuación se muestran las cifras de personas que permanecen en la Unidad de Cuidados Intensivos (UCI). 

In [8]:
df['Uci'].value_counts()

No aplica          73209
No                 24553
Si                  2250
No especificado       27
Name: Uci, dtype: int64

Se observa que un 2,25% de los pacientes se encuentran en la UCI. Siendo en total una cifra de 2.250 personas.

A continuanción se filtran solo los positivos para apreciar la importancia de estos entre el total de los pacientes en cuidados intensivos.

In [9]:
df[df['Resultado']=='Positivo']['Uci'].value_counts()

No aplica          14907
No                  8976
Si                  1015
No especificado        6
Name: Uci, dtype: int64

Se puede comprobar que los casos de UCI en circunstancias de pandemia se duplican, representando el 45,11 de ellos, siendo necesarias el doble de camas en UCI que se necesitan de forma habitual.

Se destaca que un 36,04% de los positivos en SARS CoV-2 no necesitan de cuidados intensivos.

**Número de personas que han fallecido**

En las siguientes lineas de código se extrae el número de fallecimientos de diagnoticados con SARS CoV-2 hasta la fecha.

In [10]:
df_fallecidos = df[(df['Fecha_Def'] != '9999-99-99')& \
                             (df["Resultado"] == 'Positivo')].copy()
df_fallecidos['Id_Registro'].count()

2270

El número de fallecimientos representa un 3.58% de los pacientes, con un total de 2270 fallecidos. Más adelante se profundiza sobre la tasa de fallecimientos de la pandemia.

**Número de personas que han dado positivo y sin enfermedades**

A continuación se muestra la cifra de positivos en SARS CoV-2 y sin enfermedades. Entre ellas, diabetes, enfermedad pulmonar obstructiva crónica (EPOC), asma, inmusupresión, hipertension, enfermedad cardiovascular, obesidad, enfermedad renal crónica, tabaquismo u otras.

In [11]:
def sinEnfermedad(*x):
    """Esta funcion filtra los pacientes que no tienen ninguna enfremedad. 
    Parameters
    ----------
    x : lista
        Contiene las respuestas en forma de booleano, de si tiene una enfermedad o no
    Return
    ------
    True : Boolean
    False : Boolean
    """
    if 'Si' in x:
        return False
    return True

In [12]:
lista_enfermedades = list(df.columns)
lista_enfermedades = lista_enfermedades[19:29]
positivos_sin_enf = df[(df[lista_enfermedades].apply(lambda x: sinEnfermedad(*x), axis=1)) & \
                             (df["Resultado"] == 'Positivo')]["Id_Registro"].count()

positivos_sin_enf

11985

Se obtiene un total de 11.985 casos positivos en SARS CoV-2 en pacientes que no sufren ninguna enfermedad. Esto representa el 48,12% de los positivos, no siendo significativo para contraer el virus.

**Número de personas que dan positivo con alguna enfermedad**

En el siguiente caso, se representa la cifra de positivos con alguna enfermedad previa

In [13]:
def conEnfermedad(*x):
    """Esta funcion filtra los pacientes que si tienen ninguna enfremedad. 
    Parameters
    ----------
    x : lista
        Contiene las respuestas en forma de booleano, de si tiene una enfermedad o no
    Return
    ------
    True : Boolean
    False : Boolean
    """
    if 'Si' in x:
        return True
    return False

positivos_enf = df[(df[lista_enfermedades].apply(lambda x: conEnfermedad(*x), axis=1)) & \
                             (df["Resultado"] == 'Positivo')]["Id_Registro"].count()
positivos_enf

12919

En este caso la cifra de positivos con alguna enfermedad es de 12.919, siendo una representación del 51,87%, levemente superior a la representación de pacientes positivos sin enfermedades previas.

**Número de personas que han fallecido (que han dado positivo) y sin enfermedades**

Para observar la influencia de las enfermedades  previas  se analiza la importancia de ellas en el total de fallecimientos de los positivos en SARS CoV-2.

In [17]:
falle_sin_enf = df_fallecidos[(df_fallecidos[lista_enfermedades].apply(lambda x: sinEnfermedad(*x), axis=1))]["Id_Registro"].count()

falle_sin_enf

513

Como resultado, se obtiene que el **22% de los fallecidos** tiene alguna enfermedad previa. Entre ellas: diabetes, enfermedad pulmonar obstructiva crónica (EPOC), asma, inmusupresión, hipertension, enfermedad cardiovascular, obesidad, enfermedad renal crónica, tabaquismo.
    
Representando una parte importante del total y por lo que se confirma la influencia de esta variable como riesgo.

**Tiempo promedio entre la fecha de síntomas y la fecha de defunción de los que han dado positivo**

Para el siguiente dato se analizan las fechas tanto de sintómas como de defunción con lo que se obtiene tiempo medio en el que transcure la enfermedad en los casos de fallecimiento.

In [20]:
#Cambio de formato objeto a date.time
df_fecha_sin = pd.to_datetime(df_fallecidos['Fecha_Sintomas']
                              .astype(str), format='%d/%m/%Y')
df_fecha_def = pd.to_datetime(df_fallecidos['Fecha_Def']
                              .astype(str), format='%d/%m/%Y')

promedio_def = df_fecha_def - df_fecha_sin
promedio_def.mean()

Timedelta('10 days 03:40:45.462555')

Se obtiene una media de 10 días entre la fecha de síntomas y el fallecimiento.

**Enfermedad con mayor y menor prevalencia en los pacientes que dan positivo**

En el siguiente apartado se analiza la enfermedad que más se repite entre los pacientes que son positivos y la que menos.

In [22]:
posi = df[df['Resultado']=='Positivo']

lst = []
for i in lista_enfermedades:
    x= posi[df[i]=='Si']['Id_Registro'].count()
    lst.append(x)
lst

lst2 = list(zip(lista_enfermedades, lst))

columnas = ['Patologías', 'Recuento de casos']
posi_enf = pd.DataFrame(lst2, columns= columnas)
posi_enf = posi_enf.sort_values(by=['Recuento de casos']
                                , ascending=[False]).reset_index(drop=True)

C:\Users\noeli\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [23]:
enf_mas_prevalencia = posi_enf.head(1)
enf_mas_prevalencia

,Patologías,Recuento de casos
0,Hipertension,5449


In [24]:
enf_menor_prevalencia = posi_enf.tail(1)
enf_menor_prevalencia

,Patologías,Recuento de casos
9,Inmusupr,445


Los resultados muestran que la enfermedad más destacada entre los pacientes que dan positivo es la hipertensión arterial. En la posición de menor representación se encuentran los pacientes con inmusupresión.


**Cantidad de pacientes intubados VS Cantidad de pacientes en UCI**

A continuación se realiza una comparación de las personas intubadas y las personas que requieren cuidados intensivos.

In [26]:
df[(df["Resultado"]=='Positivo') & (df['Intubado']=='Si')]['Id_Registro'].count()

1032

In [34]:
df[(df["Resultado"]=='Positivo') & (df['Uci']=='Si')]['Id_Registro'].count()

1015

En conclusión, se aprecian cifras casi idénticas por lo que se plantea analizar la correlación entre las variables en la fase de visualización.